In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import geopandas as gpd
from geopandas import GeoDataFrame
from mpl_toolkits.axes_grid1 import make_axes_locatable

# We are going to use a couple classes to manage our data and methods
import sys
sys.path.insert(1, '../src')
from classes import data, data_un_Agg
# until I integrate into class, use this for scatter plots:
from functions import scatter, plot_subgroup_hist

%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format
plt.style.use('seaborn-white')

# for testing
%load_ext autoreload
%autoreload 2

In [5]:
data = pd.read_csv('../data/prices_CBSA_vaca_construction_pop.csv')

/usr/local/anaconda3/envs/py377_maps/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.head()

,CBSA,NAME,LSAD,POPESTIMATE2019,ZIP,RegionName,State,City,Metro,CountyName,...,ErrorMargin4,cbsa_code,CSA,Name,Total,unit_1,2 Units,unit_34,unit_5,unit_for_5orMore
0,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000
1,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000
2,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000
3,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000
4,10420,"Akron, OH",Metropolitan Statistical Area,703479,44256,44256,OH,Medina,Cleveland-Elyria,Medina County,...,10.1000,"10,420.0000",184.0000,"Akron, OH ...",945.0000,856.0000,2.0000,32.0000,55.0000,1.0000


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39756 entries, 0 to 39755
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CBSA              39756 non-null  int64  
 1   NAME              39756 non-null  object 
 2   LSAD              39756 non-null  object 
 3   POPESTIMATE2019   39756 non-null  int64  
 4   ZIP               39756 non-null  int64  
 5   RegionName        39756 non-null  int64  
 6   State             39756 non-null  object 
 7   City              39756 non-null  object 
 8   Metro             39756 non-null  object 
 9   CountyName        39756 non-null  object 
 10  month_date        39756 non-null  object 
 11  house_price       39756 non-null  float64
 12  rent_price        38901 non-null  float64
 13  cbsa_title        36672 non-null  object 
 14  Q1                36672 non-null  float64
 15  ErrorMargin1      36672 non-null  float64
 16  Q2                36672 non-null  float6

In [8]:
data[data['State'] == 'AK']

,CBSA,NAME,LSAD,POPESTIMATE2019,ZIP,RegionName,State,City,Metro,CountyName,...,ErrorMargin4,cbsa_code,CSA,Name,Total,unit_1,2 Units,unit_34,unit_5,unit_for_5orMore


In [9]:
len(data['State'].sort_values().unique())

43

In [10]:
data['State'].sort_values().unique()

array(['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'MI', 'MN',
       'MO', 'MS', 'NC', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK',
       'OR', 'PA', 'RI', 'SC', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI'],
      dtype=object)